# First, let us set up a few dependencies

Don't forget to switch to a GPU-enabled colab runtime!

```
Runtime -> Change Runtime Type -> GPU
```

In [1]:
import os
import contextlib
@contextlib.contextmanager
def directory(name):
  ret = os.getcwd()
  os.chdir(name)
  yield None
  os.chdir(ret)

import subprocess
def run(input, exception_on_failure=False):
  try:
    program_output = subprocess.check_output(f"{input}", shell=True, universal_newlines=True, stderr=subprocess.STDOUT)
  except Exception as e:
    if exception_on_failure:
      raise e
    program_output = e.output

    return program_output
def prun(input, exception_on_failure=False):
  x = run(input, exception_on_failure)
  print(x)
  return x

# This mounts your google drive to this notebook. You might have to change the path to fit with your dataset folder inside your drive.

Read the instruction output by the cell bellow carefully!

In [2]:
# Create a temporary workspace
import tempfile


SESSION_WORKSPACE = tempfile.mkdtemp()
print(f"Session workspace created at: {SESSION_WORKSPACE}")

Session workspace created at: /tmp/tmpzlhfafkp


In [3]:
!ls -a "/content/drive/My Drive/"


ls: cannot access '/content/drive/My Drive/': No such file or directory


In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [5]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')
DRIVE_PATH = "/content/drive/My Drive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Unzip the dataset
import shutil
import os


DATASET_DIR_NAME = "duckietown_object_detection_dataset"
DATASET_ZIP_NAME = f"{DATASET_DIR_NAME}.zip"
DATASET_DIR_PATH = os.path.join(SESSION_WORKSPACE, DATASET_DIR_NAME)
TRAIN_DIR = "train"
VALIDATION_DIR = "val"
IMAGES_DIR = "images"
LABELS_DIR = "labels"


def show_info(base_path: str):
  for l1 in [TRAIN_DIR, VALIDATION_DIR]:
    for l2 in [IMAGES_DIR, LABELS_DIR]:
      p = os.path.join(base_path, l1, l2)
      print(f"#Files in {l1}/{l2}: {len(os.listdir(p))}")


def unzip_dataset():
  # check zipped file
  zip_path = os.path.join(DRIVE_PATH, DATASET_ZIP_NAME)
  assert os.path.exists(zip_path), f"No zipped dataset found at {zip_path}! Abort!"

  # unzip the data
  print("Unpacking zipped data...")
  shutil.unpack_archive(zip_path, DATASET_DIR_PATH)
  print(f"Zipped dataset unpacked to {DATASET_DIR_PATH}")

  # show some info
  show_info(DATASET_DIR_PATH)


unzip_dataset()

Unpacking zipped data...
Zipped dataset unpacked to /tmp/tmpzlhfafkp/duckietown_object_detection_dataset
#Files in train/images: 804
#Files in train/labels: 804
#Files in val/images: 202
#Files in val/labels: 202


In [7]:
# change  working directory to the session workspace
os.chdir(SESSION_WORKSPACE)
print(f"PWD: {os.getcwd()}")

# install pytorch and torchvision
!pip3 install torch torchvision
#!pip3 install torch==1.13.0 torchvision==0.14.0

PWD: /tmp/tmpzlhfafkp
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully unin

# Next, we will clone Yolov5

In [8]:
!rm -rf ./yolov5
!git clone https://github.com/ultralytics/yolov5.git -b v7.0
!cd yolov5 && pip3 install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17372, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 17372 (delta 42), reused 20 (delta 20), pack-reused 17313 (from 3)
Receiving objects: 100% (17372/17372), 16.25 MiB | 17.85 MiB/s, done.
Resolving deltas: 100% (11910/11910), done.
Note: switching to '915bbf294bb74c859f0b41f1c23bc395014ea679'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.5 MB/

In [16]:
!pwd
!ls -la

/tmp/tmpzlhfafkp
total 16
drwx------  4 root root 4096 Apr 10 21:05 .
drwxrwxrwt  1 root root 4096 Apr 10 21:07 ..
drwxr-xr-x  4 root root 4096 Apr 10 21:02 duckietown_object_detection_dataset
drwxr-xr-x 11 root root 4096 Apr 10 21:07 yolov5


# We now inform the training process of the format and location of our dataset

In [9]:
%%writefile ./yolov5/data/duckietown.yaml

# train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: ../duckietown_object_detection_dataset/train
val: ../duckietown_object_detection_dataset/val

# number of classes
nc: 4

# class names
names: [ 'duckie', 'cone', 'truck', 'bus' ]

Writing ./yolov5/data/duckietown.yaml


In [15]:
!ls -la yolov5

total 4296
drwxr-xr-x 11 root root    4096 Apr 10 21:07 .
drwx------  4 root root    4096 Apr 10 21:05 ..
-rw-r--r--  1 root root    7830 Apr 10 21:05 benchmarks.py
drwxr-xr-x  2 root root    4096 Apr 10 21:05 classify
-rw-r--r--  1 root root    4994 Apr 10 21:05 CONTRIBUTING.md
drwxr-xr-x  5 root root    4096 Apr 10 21:05 data
-rw-r--r--  1 root root   14083 Apr 10 21:05 detect.py
-rw-r--r--  1 root root    3701 Apr 10 21:05 .dockerignore
-rw-r--r--  1 root root   31551 Apr 10 21:05 export.py
drwxr-xr-x  8 root root    4096 Apr 10 21:07 .git
-rw-r--r--  1 root root      75 Apr 10 21:05 .gitattributes
drwxr-xr-x  4 root root    4096 Apr 10 21:05 .github
-rwxr-xr-x  1 root root    3982 Apr 10 21:05 .gitignore
-rw-r--r--  1 root root    7711 Apr 10 21:05 hubconf.py
-rw-r--r--  1 root root   35127 Apr 10 21:05 LICENSE
drwxr-xr-x  5 root root    4096 Apr 10 21:06 models
-rw-r--r--  1 root root    1554 Apr 10 21:05 .pre-commit-config.yaml
drwxr-xr-x  2 root root    4096 Apr 10 21:06 __pycac

# And we're ready to train! This step will take about 5 minutes.

Notice that we're only training for 10 epochs. That's probably not enough!

In [18]:
!ls -la yolov5


total 4296
drwxr-xr-x 11 root root    4096 Apr 10 21:07 .
drwx------  4 root root    4096 Apr 10 21:05 ..
-rw-r--r--  1 root root    7830 Apr 10 21:05 benchmarks.py
drwxr-xr-x  2 root root    4096 Apr 10 21:05 classify
-rw-r--r--  1 root root    4994 Apr 10 21:05 CONTRIBUTING.md
drwxr-xr-x  5 root root    4096 Apr 10 21:05 data
-rw-r--r--  1 root root   14083 Apr 10 21:05 detect.py
-rw-r--r--  1 root root    3701 Apr 10 21:05 .dockerignore
-rw-r--r--  1 root root   31551 Apr 10 21:05 export.py
drwxr-xr-x  8 root root    4096 Apr 10 21:07 .git
-rw-r--r--  1 root root      75 Apr 10 21:05 .gitattributes
drwxr-xr-x  4 root root    4096 Apr 10 21:05 .github
-rwxr-xr-x  1 root root    3982 Apr 10 21:05 .gitignore
-rw-r--r--  1 root root    7711 Apr 10 21:05 hubconf.py
-rw-r--r--  1 root root   35127 Apr 10 21:05 LICENSE
drwxr-xr-x  5 root root    4096 Apr 10 21:06 models
-rw-r--r--  1 root root    1554 Apr 10 21:05 .pre-commit-config.yaml
drwxr-xr-x  2 root root    4096 Apr 10 21:06 __pycac

In [17]:
!cd yolov5 && python3 train.py --cfg ./models/yolov5n.yaml --img 416 --batch 32 --epochs 10 --data duckietown.yaml --weights yolov5n.pt

2025-04-10 21:21:42.495806: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744320102.516062    6867 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744320102.522221    6867 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-10 21:21:42.543311: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [12]:
import os
import numpy as np

# List only directories that start with "exp"
all_exps = [exp for exp in os.listdir("yolov5/runs/train") if exp.startswith("exp")]

# Function to extract the experiment number (assuming "exp" alone means 0)
def extract_exp_number(exp_name):
    if exp_name == "exp":
        return 0
    try:
        # Remove the literal "exp" prefix and convert the rest to an integer.
        return int(exp_name.replace("exp", ""))
    except ValueError:
        return -1

# Build a list of experiment folders that actually have the best.pt file.
valid_exps = []
for exp in all_exps:
    best_path = os.path.join("yolov5", "runs", "train", exp, "weights", "best.pt")
    if os.path.exists(best_path):
        valid_exps.append(exp)
    else:
        print(f"Skipping {exp}: No weights file found at {best_path}")

if not valid_exps:
    print("Error: No experiment folder with a valid 'best.pt' found.")
else:
    # Extract numbers for the valid experiments and choose the one with the highest number.
    exp_numbers = [extract_exp_number(exp) for exp in valid_exps]
    latest_exp_index = np.argmax(exp_numbers)
    latest_exp = valid_exps[latest_exp_index]
    print(f"Latest valid exp is {latest_exp}")

    # Copy the file from the chosen experiment folder.
    src_path = os.path.join("yolov5", "runs", "train", latest_exp, "weights", "best.pt")
    dest_path = os.path.join("yolov5", "best.pt")
    os.system(f"cp {src_path} {dest_path}")
    print(f"Marked the model from the latest run ({latest_exp}) as {dest_path}.")


Skipping exp: No weights file found at yolov5/runs/train/exp/weights/best.pt
Error: No experiment folder with a valid 'best.pt' found.


In [73]:
import numpy as np

all_exps = os.listdir("yolov5/runs/train")
all_exps_filtered = map(lambda x: int(x.replace("exp", "1")), filter(lambda x: x.startswith("exp"), all_exps))
all_exps_filtered = np.array(list(all_exps))
latest_exp_index = np.argmax(all_exps)
latest_exp = all_exps[latest_exp_index]
print(f"Latest exp is {latest_exp}")

prun(f"cp yolov5/runs/train/{latest_exp}/weights/best.pt yolov5/best.pt")
print(f"Marked the model from the latest run ({latest_exp}) as yolov5/best.pt.")

Latest exp is exp
cp: cannot stat 'yolov5/runs/train/exp/weights/best.pt': No such file or directory

Marked the model from the latest run (exp) as yolov5/best.pt.


# Next, we can upload your model to Duckietown's cloud!

We will need our token to access our personal cloud space.

In [62]:
# TODO: Fill in the duckietown token here
YOUR_DT_TOKEN = "dt2-Npc1n4fne9txYPw32KUJ9FoCkBX4dVpPjMSdZDjNpZrCusVAmNUoaW6SSVJBNUqNw6SVUhN4rmmzSfXLyr-43dzqWFnWd8KBa1yev1g3UKnzVxZkkTbfQzdX1DLcP3mgeLMBYDBxmfeatgZ4poxsV"

Then, we chose the location of the trained model on disk and its name once uploaded to our cloud space. You should not change these values, or the robots will not be able to find the model to download.

In [63]:
import sys
sys.path.insert(0, './yolov5')

# DO NOT CHANGE THESE
model_name = "yolov5n"
model_local_path = "./yolov5/best.pt"
model_remote_path = f"courses/mooc/objdet/data/nn_models/{model_name}.pt"

# install DCSS client
!pip3 install dt-data-api

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 15.8 MB/s eta 0:00:00
  Created wheel for dt-data-api: filename=dt_data_api-2.1.0-py3-none-any.whl size=14363 sha256=b887139473b7d15f2742c8bf30c2c9314757feb9e1c448c90b0e2a7658507658
  Stored in directory: /root/.cache/pip/wheels/f9/b5/60/d35f7f09040a1b33db975be4f28603b344bf1e9d2bddef6ac7
  Created wheel for dt-authentication: filename=dt_authentication-2.2.0-py3-none-any.whl size=10095 sha256=93d8885a639af7c118dcbee636b886f00b0a609bd12f3c0f4ba2bc533dbc9561
  Stored in directory: /root/.cache/pip/wheels/f7/d5/e7/985e33386d61f6b4027fe7f9a2afec5a491972b3948b71a425
Successfully built dt-data-api dt-authentication


We now open a pointer to our cloud space and upload the model.

In [71]:
import os

if not os.path.exists(model_local_path):
    print(f"File not found: {model_local_path}")
else:
    print(f"File found at: {model_local_path}")

File not found: ./yolov5/best.pt


In [67]:
import torch
from dt_data_api import DataClient, Storage

# open a pointer to our personal duckietown cloud space
client = DataClient("dt2-Npc1n4fne9txYPw32KUJ9FoCkBX4dVpPjMSdZDjNpZrCusVAmNUoaW6SSVJBNUqNw6SVUhN4rmmzSfXLyr-43dzqWFnWd8KBa1yev1g3UKnzVxZkkTbfQzdX1DLcP3mgeLMBYDBxmfeatgZ4poxsV")
storage = client.storage("user")

# upload model
upload = storage.upload(model_local_path, model_remote_path)
upload.join()

ValueError: The file /tmp/tmpx5r8u6t2/yolov5/best.pt does not exist.

# Done!

We're done training! You can now close this tab and go back to the `Training` notebook